<a href="https://colab.research.google.com/github/shuangquan-li-con/ECON5200-37499-Applied-Data-Analytics-in-Economics/blob/main/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Load your dataset here (ensure lalonde.csv is uploaded to Colab or linked)
df = pd.read_csv('lalonde.csv')

In [2]:
df

,Unnamed: 0,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
0,1,1,37,11,1,0,1,1,0.0,0.0,9930.0460
1,2,1,22,9,0,1,0,1,0.0,0.0,3595.8940
2,3,1,30,12,1,0,0,0,0.0,0.0,24909.4500
3,4,1,27,11,1,0,0,1,0.0,0.0,7506.1460
4,5,1,33,8,1,0,0,1,0.0,0.0,289.7899
...,...,...,...,...,...,...,...,...,...,...,...
609,610,0,18,11,0,0,0,1,0.0,0.0,10150.5000
610,611,0,24,1,0,1,1,1,0.0,0.0,19464.6100
611,612,0,21,18,0,0,0,0,0.0,0.0,0.0000
612,613,0,32,5,1,0,1,1,0.0,0.0,187.6713


In [3]:
# Naive Comparison
naive_diff = df[df.treat==1]['re78'].mean()-df[df.treat==0]['re78'].mean()
print(f"Naive Difference in Means: ${naive_diff:,.2f}")
# Expected Result: -$635.03

Naive Difference in Means: $-635.03


In [4]:
# Define covariates
X = df[['age','educ','black','hispan','married','nodegree','re75','re78']]
y = df['treat']

# Fit Propensity Model
logit = LogisticRegression(solver='liblinear')
logit.fit(X, y)

# Generate Scores
df['pscore'] = logit.predict_proba(X)[:, 1]

In [9]:
from sklearn.neighbors import NearestNeighbors

# Separate groups
treated = df[df.treat==1]
control = df[df.treat==0]

# Fit NN on Control scores
nbrs = NearestNeighbors(n_neighbors=1).fit(control[['pscore']])

# Find matches for Treated scores
distances, indices = nbrs.kneighbors(treated[['pscore']])
matched_control = control.iloc[indices.flatten()]

# Construct Matched DataFrame
matched_df = pd.concat([treated, matched_control])

In [10]:
from scipy import stats

# T-test on raw data
diff = treated['re78'].mean() - control['re78'].mean()
t_stat, p_val = stats.ttest_ind(treated['re78'], control['re78'])

print(f"Raw Effect (Difference): ${diff:,.2f}")
print(f"P-value: {p_val:.4f}")


# Isolate the matched outcomes
matched_treated = matched_df[matched_df.treat==1]['re78']
matched_control = matched_df[matched_df.treat==0]['re78']

# Estimate the causal effect (T-test on matched data)
matched_diff = matched_treated.mean() - matched_control.mean()
t_stat, p_val = stats.ttest_ind(matched_treated, matched_control)


print(f"Recovered Effect (Matched Difference): ${matched_diff:,.2f}")
print(f"P-value: {p_val:.4f}")

Raw Effect (Difference): $-635.03
P-value: 0.3342
Recovered Effect (Matched Difference): $1,850.03
P-value: 0.0109


# Recovering Experimental Truths via Propensity Score Matching

## Objective
This project demonstrates the rigorous application of Propensity Score Matching to mitigate selection bias in observational studies, thereby transforming observational data failures into robust causal inferences.

## Methodology
*   **Selection Bias Modeling:** Leveraging the Lalonde dataset, a comprehensive model of selection bias was constructed to understand confounding factors.
*   **Propensity Score Estimation:** Logistic regression models were employed to estimate propensity scores, quantifying the probability of treatment assignment given observed covariates.
*   **Nearest Neighbor Matching:** A Nearest Neighbor matching algorithm was applied to pair treated individuals with control individuals exhibiting similar propensity scores, effectively balancing covariates across groups.

## Key Findings
Through the systematic application of Propensity Score Matching, this analysis successfully recovered the true average treatment effect of approximately **+$1,800**. This contrasts sharply with the heavily biased naive estimate of **-$635.03**, underscoring the critical role of causal inference techniques in yielding accurate and actionable insights from observational data.